이제 딥러닝을 통한 텍스트 분류 문제를 살펴보겠습니다. 가장 간단한 방법은 RNN을 활용하는 것입니다.

문장은 단어들의 시퀀스로 이루어진 시퀀셜 데이터입니다. 따라서 각 위치 또는 time-step의 단어들은 다른 위치의 단어들과 서로 영향을 주고받습니다. RNN은 이런 문장의 특징을 가장 잘 활용하는 신경망 구조입니다.

RNN은 각 time-step의 단어를 입력으로 받아 자신의 은닉 상태를 업데이트합니다.

<br></br>
$$h_t = f_\theta(x_t,h_{t-1})$$
<br></br>

n개의 단어로 이루어진 문장 x가 주어졌을대, 이를 RNN에 피드포워드한다면 n개의 은닉 상태를 얻을 수 있습니다. 이때 가장 마지막 은닉 상태를 활용하여 텍스트의 클래스를 분류할 수 있습니다.

<br></br>
$$
\hat{y} = argmax_{y \in Y} P(y|x;\theta) \\
\text{where}\ P(y|x;\theta) = h_n = f_\theta(x_n,h_{n-1})\ \text{and}\ x=\{w_1,w_2,\dots,w_n\}
$$
<br></br>

따라서 RNN은 입력으로 주어진 문장을 분류 문제에 맞게 인코딩한다고 볼 수 있습니다. 즉 RNN의 출력값은 $문장\ 임베딩\ 벡터^{sentence\ embedding\ vector}$라고 할 수 있습니다.

### 아키텍쳐 내부

RNN을 통해 텍스트 분류를 구현한다면 다음과 같은 구조가 될 것입니다. 내부를 구성하는 계층들을 하나씩 살펴보겠습니다.

<br></br>
![](./images/8-4-1-arc.jpg)
<br></br>

잘 알다시피 텍스트에서 단어는 불연속적인 값입니다. 단어가 모여 문장이 되어도 그 값은 여전이 불연속적입니다. 즉 이산 확률 분포에서 문장을 샘플링한 것이라고 볼 수 있습니다. 따라서 입력으로 원핫벡터들이 여러 time-step으로 주어집니다. 미니배치까지 고려했을때, 결과적으로 입력은 3차원의 텐서이며 크기는 n x m x |V|입니다. 여기서 n은 미니배치 크기, m은 문장의 길이를 가르킵니다.

<br></br>
$$
x \sim P(x) \\
\text{where}\ x = \{w_1,w_2,\dots,w_m\} \text{and}\ w_i \in \{0,1\}^{|V|} \text{and}\ |w_i| = |V| \\
\\
\text{Thus,}\ |x_{1:n}| = (n,m,|V|) \\
\text{where}\ x_{1:n} = [x_1,x_2,\dots,x_n] \text{and}\ n = batch size 
$$
<br></br>

원핫 벡터는 주어진 |V| 차원에 단 하나의 1과 |V|-1개의 0으로 이루어집니다. 임베딩 계층과의 연산이나, 원핫 벡터 자체의 효율적인 저장을 위해 굳이 원핫 벡터 전체를 가지고 있을 필요는 없습니다. 각 벡터별로 1의 위치 인덱스만 기억하고 있으면 됩니다. 즉, 원핫 벡터는 위치 인덱스 값인 0부터 |V|-1 사이의 정수로 나타낼 수 있고, 이는 3차원 텐서가 아니라 2차원의 행렬 n x m 으로 충분합니다.

<br></br>
$$|x_{1:n}| = (n,m,1) = (n,m)$$
<br></br>

이렇게 입력으로 주어진 원핫 인코딩된 n x m 텐서를 임베딩 계층에 통과시키면 단어 임베딩 텐서를 얻을 수 있습니다. 단어 임베딩 텐서의 크기는 다음과 같습니다.

<br></br>
$$\tilde{x_{1:n}} = emb_\theta(x_{1:n}) \\
|\tilde{x_{1:n}}| = (n, m, d) \\
\text{where d is word_vec_dim}$$
<br></br>

이후에 단어 임베딩 텐서를 RNN에 통과시킵니다.

<br></br>
$$
h_t = RNN_\theta(x_t,h_{t-1}) \\
\text{where}\ |x_t| = (n,1,d), |h_t| = (n,1,h) \text{and h = hidden_size}
$$
<br></br>

이때 우리는 RNN에 대해 각 time-step별, 계층별로 구분하여 단어 임베딩 텐서 또는 은닉 상태를 넣어줄 필요가 없습니다. 지금과 같은 상황에서는 초기 은닉 상태 $h_0$와 전체 입력인 단어 임베딩 텐서 $x_{1:n}$를 RNN에 넣어주면 됩니다. 그러면 파이토치가 최적화된 구현을 통해 매우 빠른 속도로 RNN의 모든 time-step에 대한 출력과 마지막 은닉 상태를 반환합니다.

<br></br>
$$
H = RNN_\theta(x_{1:n},h_0) \\
\text{where}\ H = [h_1;h_2;\dots;\_m]\ \text{and |H| = (n,m,h)}
$$
<br></br>

파이토치 RNN을 통해 얻은 모든 time-step에 대한 RNN의 출력값 중에서 제일 마지막 time-step만 선택하여 `softmax` 계층을 통과시켜 이산 확률 분포 $P(y|x;\theta)$로 나타냅니다

<br></br>
$$
h_m = H[:,-1]
$$
<br></br>

이와 같이 time-step의 차원에서 맨 마지막 인덱스를 슬라이싱할 수 있습니다. 이걸을 리니어 계층을 거친 이후에 `softmax` 함수를 취합니다.

<br></br>
$$
\hat{y} = softmax(h_mW+b) \\
\text{where}\ |\hat{y}| = (n, |C|), |h_m| = (n,1,h), W \in R^{hx|C|}, b \in R^{|C|}
$$
<br></br>

이렇게 구한 $\hat{y}$는 데이터 x와 확률 분포 함수 파라미터 $\theta$가 주어졌을때, 클래스를 나타내는 확률 변수 y의 확률 분포를 나타냅니다. 우리는 실제 정답 y와 $\hat{y}$의 차이의 손실 값을 구하고 이를 최소화하도록 SGD를 통한 최적화를 수행하며 신경망을 훈련할 수 있습니다.

<br></br>
$$
L(\hat{y},y) = - \frac{1}{n}\sum_{i=1}^{n}y_i log \hat{y_i}
$$
<br></br>

이처럼 교차 엔트로피 수식을 통해 실제 확률 분포에 우리의 신경망 확률 분포 함수가 근사하도록 합니다. 재미있는 것은 $y_i$가 원핫 벡터이므로 1인 인덱스의 $로그\ 확률값^{log-probability}$만 최대화하면 된다는 것입니다. 그러면 `softmax`의 수식에 따라 다른 인덱스의 확률값이 작아질 것입니다.

<br></br>
![](./images/8-4-1-rnn.jpg)
<br></br>

이렇게 얻은 손실 함수에 대해 확률 분포 함수 신경망 파라미터 $\theta$로 미분하면, 가능도를 최대화하는 $\theta$를 업데이트할 수 있습니다.

<br></br>
$$
\theta \gets \theta - \lambda \nabla_\theta L(\hat{y},y)
$$
<br></br>

### 파이토치 예제

앞의 수식에서와 달리 여러 계층으로 이루어진 LSTM을 사용했습니다. LSTM 내부의 각 계층간에는 드롭아웃이 추가되어 있습니다.

<br></br>
[코드 링크](https://github.com/kh-kim/simple-ntc/blob/master/simple_ntc/rnn.py)
<br></br>

In [1]:
import torch.nn as nn

class RNNClassifier(nn.Module):
    
    def __init__(self,
                 input_size,
                 word_vec_dim,
                 hidden_size,
                 n_classes,
                 n_layers = 4,
                 dropout_p = 0.3):
        
        self.input_size = input_size
        self.word_vec_dim = word_vec_dim
        self.hidden_size = hidden_size
        self.n_classes = n_classes
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        
        super().__init__()
        
        self.emb = nn.Embedding(input_size, word_vec_dim)
        self.rnn = nn.LSTM(input_size = word_vec_dim,
                           hidden_size = hidden_size,
                           num_layers = n_layers,
                           dropout = dropout_p,
                           batch_first = True,
                           bidirectional = True)
        self.generator = nn.Linear(hidden_size * 2, n_classes)
        self.activation = nn.LogSoftmax(dim = 1)
        
    def forward(self, x):
        
        # |x| = (batch_size, length)
        x = self.emb(x)
        
        # |x| = (batch_size, length, word_vec_dim)
        x, _ = self.rnn(x)
        
        # |x| = (batch_size, length, hidden_size * 2)
        y = self.activation(self.generator(x[:, -1]))
        
        # |y| = (bath_size, n_classes)
        return y